<a href="https://colab.research.google.com/github/SMKIDRaadet/SMKID_Analysis/blob/main/SMKID_Deltagere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#En undersøgelse af deltagere til S/M-KID Rådsmøder i tidens løb.

In [12]:
import pandas as pd
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

if 'S M-KID Rådet' not in os.listdir('/content/drive/My Drive/'):
  raise ValueError('S/M-KID Rådets folder not in your Google Drive.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#URLS
